## Import libraries

In [1]:
import logging
from typing import Optional, Tuple

import numpy as np
import pandas as pd
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from sklearn import metrics

In [2]:
# https://github.com/roelbertens/time-series-nested-cv/blob/master/time_series_cross_validation/custom_time_series_split.py

class CustomTimeSeriesSplit:
    def __init__(self,
                 train_set_size: int,
                 test_set_size: int
                 ):
        """
        :param train_set_size: data points (days) in each fold for the train set
        :param test_set_size: data points (days) in each fold for the test set
        """
        self.train_set_size = train_set_size
        self.test_set_size = test_set_size
        self._logger = logging.getLogger(__name__)

    def split(
        self,
        x: np.ndarray,
        y: Optional[np.ndarray] = None
    ) -> Tuple[np.ndarray, np.ndarray]:
        """Return train/test split indices.
        :param x: time series to use for prediction, shape (n_samples, n_features)
        :param y: time series to predict, shape (n_samples, n_features)
        :return: (train_indices, test_indices)
        Note: index of both x and y should be of type datetime.
        """
        if y is not None:
            assert x.index.equals(y.index)
        split_points = self.get_split_points(x)
        for split_point in split_points:
            is_train = (x.index < split_point) & (x.index >= split_point -
                                                  pd.Timedelta(self.train_set_size, unit="D"))
            is_test = (x.index >= split_point) & (x.index < split_point +
                                                  pd.Timedelta(self.test_set_size, unit="D"))
            if not is_train.any() or not is_test.any():
                self._logger.warning(
                    "Found %d train and %d test observations "
                    "skipping fold for split point %s",
                    is_train.sum(), is_test.sum(), split_point
                )
                continue
            dummy_ix = pd.Series(range(0, len(x)), index=x.index)
            ix_train = dummy_ix.loc[is_train].values
            ix_test = dummy_ix.loc[is_test].values
            if ix_train is None or ix_test is None:
                self._logger.warning(
                    "Found no data for train or test period, "
                    "skipping fold for split date %s",
                    split_point
                )
                continue
            yield ix_train, ix_test

    def get_split_points(self, x: np.array) -> pd.DatetimeIndex:
        """Get all possible split point dates"""
        start = x.index.min() + pd.Timedelta(self.train_set_size, unit="D")
        end = x.index.max() - pd.Timedelta(self.test_set_size - 1, unit="D")
        self._logger.info(f"Generating split points from {start} to {end}")
        split_range = pd.date_range(start, end, freq="D")
        first_split_point =  (len(split_range) + self.test_set_size - 1) % self.test_set_size
        return split_range[first_split_point::self.test_set_size]


class ModelBuilder:
    def __init__(self, df, target, feats, cat_feats):
        self.df = df
        self.target = target
        self.feats = feats
        self.cat_feats = cat_feats
        self.mode = "classification" if type(target) == str else "multiclassification"

    def train_folds(self, train_size=120, test_size=30, iterations=1000, early_stopping=False):
        if self.mode == "classification":
            oof_preds = np.zeros(self.df.shape[0])
        else:
            oof_preds = np.zeros((self.df.shape[0], len(targets)))

        folds_mask = np.zeros(oof_preds.shape[0])
        for fold_, (train_index, test_index) in enumerate(CustomTimeSeriesSplit(train_set_size=train_size, test_set_size=test_size).split(self.df)):
            X_train, y_train = self.df.iloc[train_index, :][self.feats], self.df.iloc[train_index, :][self.target]
            X_val, y_val = self.df.iloc[test_index, :][self.feats], self.df.iloc[test_index, :][self.target]

            weeks_train = X_train.reset_index()["dt"]
            weeks_test = X_val.reset_index()["dt"]

            tr_start_week = weeks_train.min()
            tr_end_week = weeks_train.max()
            ts_start_week = weeks_test.min()
            ts_end_week = weeks_test.max()

            print()
            print()
            print(f"Fold {fold_} train ({tr_start_week}, {tr_end_week}) test ({ts_start_week}, {ts_end_week})")

            cat_model = CatBoostClassifier(
                iterations=iterations,
                learning_rate=0.05,
                metric_period=500,
                loss_function="Logloss" if self.mode=="classification" else "MultiLogloss",
                l2_leaf_reg=10,
                eval_metric="F1" if self.mode=="classification" else "MultiLogloss", 
                task_type="CPU",
                early_stopping_rounds=100,
                random_seed=1234,
                use_best_model=early_stopping
                )

            D_train = Pool(X_train, y_train, cat_features=cat_feats, feature_names=feats)
            D_val = Pool(X_val, y_val, cat_features=cat_feats, feature_names=feats)
            
            print("Train catboost")
            cat_model.fit(
                D_train, 
                eval_set=D_val if early_stopping else None,
                verbose=True,
                plot=False
            )
            
            if self.mode == "classification":
                D_train_lgb = lgb.Dataset(X_train, y_train, weight=None, free_raw_data=False)
                D_val_lgb = lgb.Dataset(X_val, y_val, weight=None, free_raw_data=False)
                
                print("Train lgbm")
                lgbm_model = lgb.train(
                    {
                        "objective": "binary",
                        "feature_pre_filter": False,
                        "lambda_l1": 5.246525412521277e-08,
                        "lambda_l2": 3.963188589061798e-05,
                        "num_leaves": 6,
                        "feature_fraction": 0.7,
                        "bagging_fraction": 1.0,
                        "bagging_freq": 0,
                        "min_child_samples": 20,
                    },
                    D_train_lgb,
                    num_boost_round=iterations,
                    early_stopping_rounds=200 if early_stopping else None,
                    valid_sets=D_val_lgb if early_stopping else None,
                    feature_name=feats,
                    verbose_eval=500,
                )
                preds = (cat_model.predict_proba(X_val)[:, 1] + lgbm_model.predict(X_val)) / 2
                print()
                print(f"Fold {fold_} F1 Score ", metrics.f1_score(y_val, preds.round()))
                print(f"Fold {fold_} ROC AUC Score ", metrics.roc_auc_score(y_val, preds.round()))
                print(f"Fold {fold_} Confusion matrix")
                print(metrics.confusion_matrix(y_val, preds.round()))
                oof_preds[test_index] = preds
            else:
                oof_preds[test_index] = cat_model.predict(X_val)
                print(f"Fold {fold_} F1 Score ", metrics.f1_score(y_val, oof_preds[test_index].round(), average="micro"))
                try:
                    print(f"Fold {fold_} ROC AUC Score ", metrics.roc_auc_score(y_val, oof_preds[test_index]))
                except ValueError:
                    print(f"Fold {fold_} ROC AUC Score ", 0)
                    
            folds_mask[test_index] = 1
        
        if self.mode == "classification":
            oof_f1micro = metrics.f1_score(self.df.iloc[folds_mask == 1, :][self.target], oof_preds[folds_mask == 1].round(), average="micro")
            oof_f1micro = metrics.roc_auc_score(self.df.iloc[folds_mask == 1, :][self.target], oof_preds[folds_mask == 1], average="micro")
        else:
            oof_f1micro = metrics.f1_score(self.df.iloc[folds_mask == 1, :][self.target], oof_preds[folds_mask == 1].round(), average="micro")
            oof_f1micro = metrics.roc_auc_score(self.df.iloc[folds_mask == 1, :][self.target], oof_preds[folds_mask == 1], average="micro")
        
        print()
        print("Overall OOF F1 Micro ", oof_f1micro)
        print("Overall OOF Mean ROC AUC Score ", oof_f1micro)
        
    def train_final_models(self, iterations=1000, early_stopping=False):
        if self.mode == "classification":
            X_train, y_train = self.df.iloc[:, :][self.feats], self.df.iloc[:, :][self.target]

            cat_model = CatBoostClassifier(
                iterations=iterations,
                learning_rate=0.05,
                metric_period=500,
                loss_function="Logloss",
                l2_leaf_reg=10,
                eval_metric="F1", 
                task_type="CPU",
                random_seed=1234,
                use_best_model=early_stopping
                )

            D_train = Pool(X_train, y_train, cat_features=cat_feats, feature_names=feats)

            print("Train catboost")
            cat_model.fit(
                D_train, 
                eval_set=None,
                verbose=True,
                plot=False
            )

            D_train_lgb = lgb.Dataset(X_train, y_train, weight=None, free_raw_data=False)

            print("Train lgbm")
            lgbm_model = lgb.train(
                {
                    "objective": "binary",
                    "feature_pre_filter": False,
                    "lambda_l1": 5.246525412521277e-08,
                    "lambda_l2": 3.963188589061798e-05,
                    "num_leaves": 6,
                    "feature_fraction": 0.7,
                    "bagging_fraction": 1.0,
                    "bagging_freq": 0,
                    "min_child_samples": 20,
                },
                D_train_lgb,
                num_boost_round=iterations,
                valid_sets=None,
                feature_name=feats,
                verbose_eval=500,
            )

            return cat_model, lgbm_model

        elif self.mode == "multiclassification":
            raise NotImplementedError 

## Prepare training data

In [3]:
df = pd.read_csv("merged.csv")

In [4]:
df["SATELLITE"].unique()

array(['Terra', 'Aqua'], dtype=object)

In [5]:
df["CONFIDENCE"] = df["CONFIDENCE"].map({"l":0, "h":1, "n":3})
df["SATELLITE"] = df["SATELLITE"].map({"1":0, "N":1})
df["DAYNIGHT"] = df["DAYNIGHT"].map({"D":0, "N":1})
df["dt"] = pd.to_datetime(df["dt"]).dt.date
df = df.set_index("dt")

In [6]:
targets = ["infire_day_1","infire_day_2","infire_day_3","infire_day_4","infire_day_5","infire_day_6","infire_day_7","infire_day_8"]
feats = ["BRIGHTNESS","SCAN","TRACK","ACQ_TIME","SATELLITE","DAYNIGHT","CONFIDENCE","BRIGHT_T31","FRP"]
#cat_feats = ["grid_index", "DAYNIGHT","SATELLITE"]
cat_feats = []

In [7]:
targets = ["infire_day_1","infire_day_2","infire_day_3","infire_day_4","infire_day_5","infire_day_6","infire_day_7","infire_day_8"]
df["target"] = (df[targets].sum(axis=1)>0).astype(np.uint8)

In [8]:
df["target"].value_counts(normalize=True)

1    0.852313
0    0.147687
Name: target, dtype: float64

In [9]:
### syntetic data
DROPOUT_PROBA = 0.7
UPSAMPLE_RATE = 6

df_syn_base = df[df["target"]==0][feats]
df_syn_final = pd.DataFrame()

for i in range(UPSAMPLE_RATE):
    df_syn = df_syn_base.copy()
    for f in feats[3:]:
        df_syn[f] = df_syn[f].apply(lambda x: x if np.random.random()>DROPOUT_PROBA else None).sample(frac=1.0).values
    df_syn_final = pd.concat([df_syn_final, df_syn], axis=0)

df_syn_final["target"] = 0

In [10]:
df_combined = pd.concat([
    df[feats+["target"]],
    df_syn_final], axis=0)

In [11]:
df_combined["target"].value_counts(normalize=True)

0    0.548113
1    0.451887
Name: target, dtype: float64

## Train with single lable (will we see fire during a period of 8 days)

In [12]:
fire_model = ModelBuilder(df_combined, "target", feats, cat_feats)

In [13]:
fire_model.train_folds(train_size=120, test_size=30, iterations=1000, early_stopping=False)

/home/imd/miniconda3/envs/wildfires/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)




Fold 0 train (2020-02-20, 2020-06-18) test (2020-06-19, 2020-07-18)
Train catboost
0:	learn: 0.9088065	total: 58.8ms	remaining: 58.7s
500:	learn: 0.9158019	total: 1.32s	remaining: 1.32s
999:	learn: 0.9267353	total: 2.57s	remaining: 0us
Train lgbm
[LightGBM] [Info] Number of positive: 6978, number of negative: 9289
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000415 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1079
[LightGBM] [Info] Number of data points in the train set: 16267, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.428967 -> initscore=-0.286069
[LightGBM] [Info] Start training from score -0.286069


/home/imd/miniconda3/envs/wildfires/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "



Fold 0 F1 Score  0.8901392860437228
Fold 0 ROC AUC Score  0.9044177170846646
Fold 0 Confusion matrix
[[5731  926]
 [ 265 4825]]


Fold 1 train (2020-03-21, 2020-07-18) test (2020-07-19, 2020-08-17)
Train catboost
0:	learn: 0.9029634	total: 3.1ms	remaining: 3.09s


/home/imd/miniconda3/envs/wildfires/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001172 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 680
[LightGBM] [Info] Number of data points in the train set: 865, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.320231 -> initscore=-0.752709
[LightGBM] [Info] Start training from score -0.752709

Fold 8 F1 Score  0.8228571428571428
Fold 8 ROC AUC Score  0.8536427275084262
Fold 8 Confusion matrix
[[117  16]
 [ 15  72]]


Fold 9 train (2020-11-16, 2021-03-15) test (2021-03-16, 2021-04-14)
Train catboost
0:	learn: 0.8662420	total: 563us	remaining: 563ms


/home/imd/miniconda3/envs/wildfires/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


500:	learn: 0.9931507	total: 207ms	remaining: 206ms
999:	learn: 1.0000000	total: 412ms	remaining: 0us
Train lgbm
[LightGBM] [Info] Number of positive: 145, number of negative: 252
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004959 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 327
[LightGBM] [Info] Number of data points in the train set: 397, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.365239 -> initscore=-0.552695

/home/imd/miniconda3/envs/wildfires/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "



[LightGBM] [Info] Start training from score -0.552695

Fold 9 F1 Score  0.7746031746031746
Fold 9 ROC AUC Score  0.7998847677793457
Fold 9 Confusion matrix
[[1202  156]
 [ 341  854]]

Overall OOF F1 Micro  0.9307062909109078
Overall OOF Mean ROC AUC Score  0.9307062909109078


In [14]:
cat_model, lgbm_model = fire_model.train_final_models()

Train catboost
0:	learn: 0.9153127	total: 4.56ms	remaining: 4.56s
500:	learn: 0.9181814	total: 2.76s	remaining: 2.75s
999:	learn: 0.9208589	total: 4.98s	remaining: 0us
Train lgbm
[LightGBM] [Info] Number of positive: 21278, number of negative: 25809
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000357 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1079
[LightGBM] [Info] Number of data points in the train set: 47087, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.451887 -> initscore=-0.193050
[LightGBM] [Info] Start training from score -0.193050


/home/imd/miniconda3/envs/wildfires/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


### Save models

In [15]:
cat_model.save_model("catboost", format="cbm")

lgbm_model.save_model("light_gbm.txt")